# ShorelineMonitor Shorelines

The ShorelineMonitor dataset provides Satellite-Derived Shorelines (SDS) extracted from annually
composited Landsat satellite imagery spanning the years 1984-2024. These shorelines offer a global
view of coastal change and shoreline dynamics, serving as a critical foundation for global coastal
monitoring and analytics.

The shorelines are represented as geospatial line features. Additional attributes in the dataset
include measures of curvature (sinuosity, fractal dimension and self-intersection density), composite
image identifiers, coverage metrics, and parameters used in the shoreline extraction process.

This dataset builds on earlier efforts by Luijendijk et al. (2018) and Deltares, and has now been
optimized as a cloud-native dataset (Calkoen et al. 2025) to facilitate global-scale coastal analytics.

The dataset is currently a work in progress by Deltares (RPC) and is available upon reasonable request.
Please contact the data provider for more information or collaboration opportunities.


In [ ]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}


coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
shorelines_col = coclico_catalog.get_child("shorelinemonitor-shorelines")

In [ ]:
shorelines_extents = read_snapshot(shorelines_col, storage_options=storage_options)
shorelines_extents.head()

In [ ]:
shorelines_extents.explore()

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = (43.32, -1.97)
m.zoom = 14
m.layout.height = "800px"
m

In [ ]:
west, south, east, north = m.west, m.south, m.east, m.north

# Default values for west, south, east, north (useful for testing or when not set)
if not west:
    west, south, east, north = (4.796, 53.108, 5.229, 53.272)

# Calculate center
center_lon = (west + east) / 2
center_lat = (north + south) / 2

# Print center in copy-pasteable format (rounded to 2 decimal places)
print(f"m.center = ({center_lon:.2f}, {center_lat:.2f})")

# Print extent in copy-pasteable format (rounded to 3 decimal places)
print(f"west, south, east, north = ({west:.3f}, {south:.3f}, {east:.3f}, {north:.3f})")

roi = gpd.GeoDataFrame(
    geometry=[shapely.geometry.box(west, south, east, north)], crs=4326
)

## Read data from storage

In [ ]:
import coastpy

sds_engine = coastpy.io.STACQueryEngine(
    stac_collection=shorelines_col,
    storage_backend="azure",
    # columns = ["geometry", "shoreline_id", "datetime", "bbox", ] ... # when you don't need all data
)

In [ ]:
shorelines = sds_engine.get_data_within_bbox(
    west, south, east, north, sas_token=sas_token
)
shorelines.head()

## Plot the data on a map

In [ ]:
shorelines = gpd.overlay(shorelines, roi)

shorelines = shorelines.assign(
    year=pd.to_datetime(shorelines.datetime).dt.strftime("%Y")
)
shorelines["year"] = shorelines["year"].astype("int16")

In [ ]:
import colorcet as cc
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Create a figure and axis
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

cmap = ListedColormap(cc.bmy)

# Plot the LineString geometries colored by 'year'
shorelines.plot(
    ax=ax,
    column="year",  # Attribute to color by
    cmap=cmap,  # Colormap
    legend=True,  # Enable legend
    legend_kwds={"label": "Year", "orientation": "vertical"},
    linewidth=1.5,  # Line width for better visibility
    alpha=0.8,  # Transparency
)

import contextily as ctx

ctx.add_basemap(
    ax, crs=shorelines.crs.to_string(), source=ctx.providers.Esri.WorldImagery
)

# Show the plot
plt.tight_layout()
plt.show()

## Work with the full dataset

In [ ]:
import dask_geopandas

fs = fsspec.filesystem("az", **storage_options)
urlpaths = shorelines_extents.href.to_list()
shorelines = dask_geopandas.read_parquet(urlpaths, filesystem=fs)

In [ ]:
shorelines